In [1]:
import HYTRAN as ht

import configparser
import numpy as np
import pandas as pd
import pint_pandas
import plotly.express as px
from spectral import EcostressDatabase

In [2]:
# Initialize ECOSTRESS db
# http://www.spectralpython.net/libraries.html

db = EcostressDatabase("./datasets/ecostress.db")

# Pulls parameters of interest from config file. Just for development/ debugging.
config = configparser.ConfigParser()
config.read('./config.ini')

tgt_params = dict(config.items('target_properties'))
bb_temp = float(tgt_params['blackbody_temp'])
emm = 1 - float(tgt_params['uniform_albedo'])

inst_params = dict(config.items('instrument_properties'))
f_no = float(inst_params['f_number'])
fl_mm = float(inst_params['focal_length_mm'])
lens_trans = float(inst_params['broadband_transmission'])
pp_um = float(inst_params['pixel_pitch_um'])
d_star = float(inst_params['detectivity'])
t_int_us = float(inst_params['integration_time_usec'])

# TO BE ADDED TO CONFIG
band1_cuton = 10.6
band1_cutoff = 11.19
band2_cuton = 11.5
band2_cutoff = 12.51

scenario_params = dict(config.items('scenario_properties'))
wl_step_um = float(scenario_params['wavelength_step_um'])

NameError: name '__file__' is not defined

In [3]:
scene = ht.Scene()

density = scene.gen_spectral_radiant_density(wav=20, T=bb_temp)
density*ht.ureg.micrometer # - THIS WORKS NOW!! Need to point to the ht.ureg object to do unit math here.

1.5600373398761872e-07 <Unit('joule / meter ** 3')>

In [4]:
spectral_power, wav = scene.gen_spectral_power(wav=20, T=bb_temp)
spectral_power
# power.to(ureg.watt/ ureg.meter**2 / ureg.micrometer)

3.721738304923561 <Unit('watt / meter ** 2 / micrometer')>

In [5]:
wavelengths = np.arange(.1, 25, wl_step_um)

temps = [250, 275, 300, 325, 350]
vals = {}

for T in temps:
    spectral_powers, wavs = scene.gen_spectral_power(wav=wavelengths, T=T)
    vals[T] = spectral_powers.to_tuple()[0]

df = pd.DataFrame.from_dict(vals)
df['wav_um'] = wavelengths

In [6]:
px.line(df.reset_index(), x='wav_um', y=[250, 275, 300, 325, 350], log_y=False, log_x=False,).update_layout(
    title_text = "Blackbody Emitted Spectral Radiance",
    xaxis_title = "Wavelength[um]",
    yaxis_title = "Spectral Radiance [W / m^2 / um]",
    legend_title = "Temperature [K]",
)

In [7]:
## Integrates the radiance of the last curve generated (350 K) in above plot

radiant_ext = scene.integrate_spectral_intensity(wavs, spectral_powers, [11,11.1]) # waveband not working yet
radiant_ext

238.90999195731837 <Unit('watt / meter ** 2')>

In [8]:
inst = ht.Instrument(f_no=f_no, bb_trans=lens_trans, 
                        fl_mm = fl_mm, kind="lens")
inst.add_detector(name="Pico1024", pp_um=pp_um, 
                    t_int_us=t_int_us, detectivity=d_star)
inst.calc_NEP()

1.939850724497463e-13 <Unit('watt')>

In [9]:
scenario = ht.Scenario()

rx_pwr_1 = scenario.rx_detector_power_greybody(inst, bb_temp, 
            wl1=band1_cuton, wl2=band1_cutoff, emmissivity=emm)
rx_pwr_2 = scenario.rx_detector_power_greybody(inst, bb_temp + 1, 
            wl1=band1_cuton, wl2=band1_cutoff, emmissivity=emm)
oneK_pwr_10to11 = rx_pwr_2 - rx_pwr_1
oneK_pwr_band1 = oneK_pwr_10to11

# rx_pwr_1.to(ht.ureg.watt)
oneK_pwr_band1.to(ht.ureg.watt)

2.1981397953177575e-12 <Unit('watt')>

In [10]:
dPdT = oneK_pwr_10to11 / (1*ht.ureg.kelvin)
NEDT = inst.NEP / dPdT 
NEDT.to(ht.ureg.kelvin)

0.08824965221181681 <Unit('kelvin')>

In [11]:
def calc_NEDT(inst, wl1, wl2):
    rx_pwr_1 = scenario.rx_detector_power_greybody(inst, bb_temp, wl1=wl1, wl2=wl2, emmissivity=emm)
    rx_pwr_2 = scenario.rx_detector_power_greybody(inst, bb_temp + 1, wl1=wl1, wl2=wl2, emmissivity=emm)
    oneK_pwr_10to11 = rx_pwr_2 - rx_pwr_1

    rx_pwr_1.to(ht.ureg.watt)

    dPdT = oneK_pwr_10to11 / (1*ht.ureg.kelvin)
    NEDT = inst.NEP / dPdT 
    NEDT = NEDT.to(ht.ureg.kelvin)
    return NEDT

In [12]:
def update_inst(f_no=f_no, detectivity=d_star, t_int_us=t_int_us):
    inst = ht.Instrument(f_no=f_no, bb_trans=lens_trans, fl_mm = fl_mm, kind="lens")
    inst.add_detector(name="Pico1024", pp_um=pp_um, t_int_us=t_int_us, detectivity=detectivity)
    NEP = inst.calc_NEP()
    return inst

In [13]:
inst = update_inst()
calc_NEDT(inst, wl1=10, wl2=11)

0.04946295358057414 <Unit('kelvin')>

In [14]:
d_star

80000000000.0

In [15]:
NEDT_df = pd.DataFrame()

f_nos = [1, 1.2, 1.4, 1.6, 1.8]
for f in f_nos:
    NEDTs = {}
    inst = update_inst(f_no=f, detectivity=8e10)
    NEDTs['band_1'] = calc_NEDT(inst, wl1=band1_cuton, wl2=band1_cutoff).to_tuple()[0]
    NEDTs['band_2'] = calc_NEDT(inst, wl1=band2_cuton, wl2=band2_cutoff).to_tuple()[0]
    NEDTs['broadband'] = calc_NEDT(inst, wl1=10, wl2=12.5).to_tuple()[0]
    NEDT_df = NEDT_df.append(NEDTs, ignore_index=True)

NEDT_df['f_no'] = f_nos
# pd.DataFrame(NEDTs, index=f_nos)
display(NEDT_df)
display(NEDT_df.set_index('f_no'))

,band_1,band_2,broadband,f_no
0,0.045025,0.030913,0.011154,1.0
1,0.064836,0.044515,0.016061,1.2
2,0.088250,0.060590,0.021861,1.4
3,0.115265,0.079137,0.028554,1.6
4,0.145882,0.100158,0.036138,1.8


,band_1,band_2,broadband
f_no,,,
1.0,0.045025,0.030913,0.011154
1.2,0.064836,0.044515,0.016061
1.4,0.088250,0.060590,0.021861
1.6,0.115265,0.079137,0.028554
1.8,0.145882,0.100158,0.036138


In [16]:
px.line(NEDT_df, x='f_no', y=['band_1', 'band_2', 'broadband']).update_layout(
    title_text = "Noise Equivalent Temperature Difference (NEDT) vs. F-number",
    xaxis_title = "Optical System F-number",
    yaxis_title = "NEDT [Kelvin]",
    legend_title = "Band",
)

In [17]:
d_stars = [8e8, 2e9, 8e9, 2e10, 8e10, 2e11, 8e11]

NEDT_df = pd.DataFrame()

for d in d_stars:
    NEDTs = {}
    inst = update_inst(f_no=f, detectivity=d)
    NEDTs['band_1'] = calc_NEDT(inst, wl1=band1_cuton, wl2=band1_cutoff).to_tuple()[0]
    NEDTs['band_2'] = calc_NEDT(inst, wl1=band2_cuton, wl2=band2_cutoff).to_tuple()[0]
    NEDTs['broadband'] = calc_NEDT(inst, wl1=10, wl2=12.5).to_tuple()[0]
    NEDT_df = NEDT_df.append(NEDTs, ignore_index=True)

NEDT_df['detectivity'] = d_stars
# pd.DataFrame(NEDTs, index=f_nos)
display(NEDT_df.set_index('detectivity'))

,band_1,band_2,broadband
detectivity,,,
8.000000e+08,14.588208,10.015834,3.613821
2.000000e+09,5.835283,4.006334,1.445528
8.000000e+09,1.458821,1.001583,0.361382
2.000000e+10,0.583528,0.400633,0.144553
8.000000e+10,0.145882,0.100158,0.036138
2.000000e+11,0.058353,0.040063,0.014455
8.000000e+11,0.014588,0.010016,0.003614


In [18]:
px.line(NEDT_df, x='detectivity', y=['band_1', 'band_2', 'broadband'], log_x=True, log_y=True).update_layout(
    title_text = "Noise Equivalent Temperature Difference (NEDT) vs. Detectivity",
    # subtitle_text = "test",
    xaxis_title = "Detectivity [Jones]",
    yaxis_title = "NEDT [Kelvin]",
    legend_title = "Band",
)

In [19]:
t_ints = [12, 14, 16, 18, 20, 24, 30, 40, 50, 60]

NEDT_df = pd.DataFrame()

for t in t_ints:
    NEDTs = {}
    inst = update_inst(f_no=1.4, detectivity=8e10, t_int_us=t)
    NEDTs['band_1'] = calc_NEDT(inst, wl1=band1_cuton, wl2=band1_cutoff).to_tuple()[0]
    NEDTs['band_2'] = calc_NEDT(inst, wl1=band2_cuton, wl2=band2_cutoff).to_tuple()[0]
    NEDTs['broadband'] = calc_NEDT(inst, wl1=10, wl2=12).to_tuple()[0]
    NEDT_df = NEDT_df.append(NEDTs, ignore_index=True)

NEDT_df['t_int'] = t_ints
# pd.DataFrame(NEDTs, index=f_nos)
display(NEDT_df.set_index('t_int'))

,band_1,band_2,broadband
t_int,,,
12,0.088250,0.060590,0.026433
14,0.081703,0.056095,0.024472
16,0.076426,0.052472,0.022892
18,0.072056,0.049471,0.021582
20,0.068358,0.046933,0.020475
24,0.062402,0.042843,0.018691
30,0.055814,0.038320,0.016718
40,0.048336,0.033186,0.014478
50,0.043233,0.029683,0.012949


In [20]:
px.line(NEDT_df, x='t_int', y=['band_1', 'band_2', 'broadband'], log_x=False, log_y=False).update_layout(
    title_text = "Noise Equivalent Temperature Difference (NEDT) vs. Integration Time",
    # subtitle_text = "test",
    xaxis_title = "Integration Time [milliseconds]",
    yaxis_title = "NEDT [Kelvin]",
    legend_title = "Band",
)

In [21]:
pwr_df = pd.Series(scene.ext_power_bb[1], index=scene.ext_power_bb[0])
scene.ext_power_bb[1].max()

/Users/kevinlalli/.virtualenvs/pyspectral/lib/python3.9/site-packages/numpy/core/_asarray.py:102: UnitStrippedWarning:

The unit of the quantity is stripped when downcasting to ndarray.

/Users/kevinlalli/.virtualenvs/pyspectral/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1302: UnitStrippedWarning:

The unit of the quantity is stripped when downcasting to ndarray.

/Users/kevinlalli/.virtualenvs/pyspectral/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1672: UnitStrippedWarning:

The unit of the quantity is stripped when downcasting to ndarray.



21.510929517180532 <Unit('watt / meter ** 2 / micrometer')>

In [22]:
pwr_df

0.10     3.519972e-166
0.15     1.499267e-107
0.20      2.023408e-78
0.25      4.725765e-61
0.30      1.515392e-49
             ...      
24.75     3.007570e+00
24.80     2.989712e+00
24.85     2.971981e+00
24.90     2.954376e+00
24.95     2.936895e+00
Length: 498, dtype: float64

In [23]:
px.line(pwr_df)

In [24]:
# http://www.spectralpython.net/libraries.html

s = db.get_signature(579)
px.line(x=s.x, y=s.y).update_layout(
    title_text=s.sample_name,
    xaxis_title = 'Wavelength [microns]',
    yaxis_title = 'Reflectance [%]'
)

In [25]:
from scipy import interpolate

In [26]:
spectral_function = interpolate.interp1d(s.x, s.y)
len(wavs)

498

In [27]:
len(spectral_powers)

498

In [28]:
s_resamp = spectral_function(wavs[50:300])

In [29]:
px.line(x = wavs[50:300], y = s_resamp)

In [30]:
pwr_df_crop = pd.DataFrame(pwr_df.iloc[50:300], columns=['bb_spectrum'])

In [31]:
pwr_df_crop

,bb_spectrum
2.60,0.136316
2.65,0.167009
2.70,0.202726
2.75,0.243942
2.80,0.291131
...,...
14.85,11.046708
14.90,10.970834
14.95,10.895439
15.00,10.820524


In [32]:
pwr_df_crop['reflectance'] = s_resamp
pwr_df_crop['emissivity'] = 100 - s_resamp
pwr_df_crop['emit_spectrum'] = (pwr_df_crop.emissivity/100)*pwr_df_crop.bb_spectrum
pwr_df_crop.index.name = "wavelength"

In [33]:
pwr_df_crop

,bb_spectrum,reflectance,emissivity,emit_spectrum
wavelength,,,,
2.60,0.136316,11.949000,88.051000,0.120027
2.65,0.167009,11.749998,88.250002,0.147386
2.70,0.202726,11.305001,88.694999,0.179808
2.75,0.243942,9.943500,90.056500,0.219686
2.80,0.291131,8.625002,91.374998,0.266021
...,...,...,...,...
14.85,11.046708,10.004790,89.995210,9.941508
14.90,10.970834,10.237256,89.762744,9.847722
14.95,10.895439,10.065928,89.934072,9.798712


In [34]:
fig = px.line(pwr_df_crop[(pwr_df_crop.index > 2) & (pwr_df_crop.index < 12.5)].reset_index(), width=900,
x="wavelength", y=["emit_spectrum", "bb_spectrum", "emissivity"], facet_col="variable", facet_col_wrap=2)
# x="index", y=["emissivity", "bb_spectrum", "emit_spectrum"])
fig.update_yaxes(title="Exitant Power [W/m^2]", row =2, col=1)
fig.update_yaxes(title="Emissivity [%]", row=1, col=1)
fig.update_yaxes(matches=None)
fig.update_yaxes(showticklabels=True, col=2) 


fig.add_vrect(x0=band1_cuton, x1=band1_cutoff, col=1,
              annotation_text="TIR1", annotation_position="top left",
              fillcolor="green", opacity=0.25, line_width=0)

fig.add_vrect(x0=band2_cuton, x1=band2_cutoff, col=1,
              annotation_text="TIR2", annotation_position="top left",
              fillcolor="blue", opacity=0.25, line_width=0)

# wide_df = px.data.medals_wide(indexed=False)
# fig = px.bar(wide_df, x="nation", y=["gold", "silver", "bronze"], facet_col="variable", color="nation")
# fig.show()